```python
output = relu(dot(W, input) + b)
```

* 이 식에서 Tensor W와 b는 층의 속성처럼 볼 수 있음. **가중치 (weight)** 또는 **훈련되는 파라미터 (trainable parameter)** 라고 부름. 이런 가중치에 훈련 데이터를 신경망에 노출시켜서 학습된 정보가 담겨 있음.
* 초기에는 가중치 행렬이 작는 난수로 채워져있고 이를 **무작위 초기화 (Random Initialization)** 단계라고 부름.
* `W`와 `b`가 난수일때 `relu(dot(W, input) + b)`가 유용한 표현을 만들 것이라고 기대할 수 없음. 즉, 의미없는 표현이 만들어짐
* 피드백 신호에 기초하여 가중치가 점진적으로 조정됨.
* 이런 점진적인 조정 또는 **Training**이 머신 러닝 학습의 핵심

훈련은 다음과 *Traning Loop*안에서 일어남
1. 훈련 샘플 x와 이에 상응하는 타깃 y의 배치를 추출
2. x를 사용하여 네트워크를 실행하고, 예측 `y_pred`를 구함
3. `y_pred`와 `y`의 차이를 측정하여 배치에 대한 네트워크의 손실을 계산
4. 배치에 대한 손실이 조금 감소되도록 네트워크의 모든 가중치를 업데이트

결국 훈련 데이터에서 네트워크의 손실, 즉 예측 `y_pred`와 타겟 `y`의 오차가 매우 작아질 것

가장 어려운 부분은 가중치를 업데이트하는 4단계.
* 가중치 값이 증가해야할지, 감소해야할지, 또 얼만큼 업데이트해야 할지?


한가지 방법은 네트워크 가중치 행렬의 원소를 모두 고정하고 관심있는 하나의 값을 적용해보는 것
- 가중치의 초깃값이 0.3이라고 가정
- 배치 데이터를 Forward Pass에 통과시킨 후 네트워크의 손실이 0.5가 나옴
- 이 가중치 값을 0.35로 변경하고 손실이 0.6으로 증가
- 반대로 가중치 값을 0.25로 변경하고 손실이 0.4로 감소
- 이 경우에는 가중치를 -0.05만큼 업데이트한 것이 손실을 줄이는데 기여함
- 이런 방식으로 네트워크의 모든 가중치에 반복
- 이 접근 방식은 모든 가중치 행렬의 원소마다 두번의 정방향 패스를 계산해야 하므로 매우 비효율적

신경망에 사용되는 모든 연산은 미분 가능 *differentiable*가 가능하다는 장점을 사용하여 네트워크 가중치에 대한 손실의 gradient를 계산하는 것이 더 좋은 방법
- gradient의 반대 방향으로 가중치를 이동하면 손실이 감소 됨

### 2.4.1 Gradient

`Gradient`는 Tensor Operation의 `Derivative`이다.

입력 벡터 `x`, 행렬 `W`, 타겟 `y`과 손실 함수 `loss`가 있다고 가정할때, `W`를 사용하여 타겟의 예측 `y_pred`를 계산하고 손실 (`y_pred`과 `y`의 오차)를 계산할 수 있음

```python
y_pred = dot(W, x)
loss_value = loss(y_pred, y)
```

입력 데이터 `x`과 `y`가 고정되어있다면 이 함수는 `W`의 손실 값을 매핑하는 함수로 표현할 수 있음.
```python
loss_value = f(W)
```

W의 현재 값을 W0라고 할때, W0에서 f의 변화율은 W와 같은 크기의 텐서인 `gradient(f)(W0)`라고 할 수 있음. 이 텐서의 각 원소 `gradient(f)(W0)[i, j]`는 `W0[i,j]`를 변경했을때 `loss_value`가 바뀌는 방향과 크기를 나타냄. 다시 말해서 `gradient(f)(W0)`가 `W0`에서 함수 `f(W) = loss_value`의 gradient임

함수 f(x)의 derivative는 f의 기울기로 해석할 수 있듯, `gradient(f)(W0)`는 W0에서 `f(w)`의 기울기를 나타내는 텐서로 해석할 수 있음. 그렇기때문에 함수 `f(x)`에 대해서는 변화율의 반대 방향으로 `x`을 움직이면 `f(x)`의 값을 감소시킬 수 있음. 동일한 방식을 적용하면 함수 `f(W)`입장에서는 `gradient`의 방대 방향으로 `W`로 움직이면 `f(W)`의 값을 줄일 수 있음

### 2.4.2 Stochastic Gradient Descent (확률적 경사 하강법)

미분이 가능한 함수 *differentiable function*가 주어지면 이론적으로 이 함수의 최솟값을 구할 수 있음. 함수의 최솟값은 `derivative = 0`인 지점이므로 `derivative`가 0인 지점을 모두 찾아내고 이 중에서 어떤 포인트의 함수 값이 가장 작은지 확인.

신경망에 적용한다면 **가장 작은** `손실 함수의 값`을 만드는 가중치의 조합을 찾는 것을 의미함.
* gradient(f)(W) = 0를 풀어서 해결
* N개의 변수로 이루어진 다항식
* N은 네트워크의 가중치의 개수

이 개념을 사용하여 서술하였던 훈련 과정을 더 효율적으로 구현할 수 있음
1. 훈련 샘플 x와 이에 상응하는 타깃 y의 배치를 추출
2. x를 사용하여 네트워크를 실행하고, 예측 `y_pred`를 구함
3. `y_pred`와 `y`의 차이를 측정하여 배치에 대한 네트워크의 손실을 계산
4. 네트워크의 `parameter`에 대한 손실 함수의 gradient를 계산 (<- Backward Pass)
5. 그래디언트의 반대 방향으로 파라미터를 조금 이동시킴.
    * 예를 들면 `W -= step * gradient`를 하면 배치에 대한 손실이 조금 감소됨
    
이 과정을 **mini-batch stochastic gradient descent (mini-batch SGD)** 라고 부름.
* **Stochastic** (확률적): 각 배치 데이터가 무작위로 선택된다는 의미

![2-4](./images/2-4-2-sgd1.png)

그림에서 보이듯 step값을 적절히 고르는게 중요함. 
* 이 값이 너무 작으면 곡선을 따라 내려가는데 너무 많은 반복이 필요하고, `local minimum`에 갇힐 수 있음.
* step이 너무 크면 곡선에서 완전 임의의 위치로 이동할 수도 있음

극단적인 SGD 예시:

* (True) SGD: Mini-batch SGD의 변종으로 반복마다 하나의 샘플과 하나의 타겟을 뽑는 것.
* Batch SGD: 가용한 모든 데이터를 사용하여 반복함. 더 정확하게 업데이트 하지만 더 많은 비용이 듦.

두 극단적인 방법의 효율적인 절충안은 적절한 크기의 *미니 배치*를 사용하는 것

업데이트할 다음 가중치를 계산할때 현재 그래디언트 값만 보지않고 이전에 업데이트된 가중치를 여러 가지 다른 방식으로 고려하는 SGD의 변종이 있음
* 모멘텀을 사용한 SGD, Adagrad, RMSProp등

이런 변종들 모두 **optimization method** 또는 **optimizer**라고 부름.
* 이런 변종에서 사용되는 **momentum**이라는 개념은 SGD의 문제점인 수렴 속도와 local minima 를 
* 물리학에서 영감받은 방식: 모멘텀이 충분하면 공이 골짜기에 갇히지 않고 전역 최솟값에 도달
* 모멘텀은 *현재 기울기 값*뿐만 아니라 *현재 속도*를 함께 고려하여 각 단계에서 공을 움직임
* 실전에 적용할 때는 그래디언트 값뿐만 아니라 이전에 업데이트한 파라미터에 기초하여 `w`를 업데이트함.

```
past_velocity = 0.
momentum = 0.1 
while loss > 0.01:
    w, loss, gradient = get_current_parameters()
    velociy = momentum * past_velocity - learning_rate * gradient
    w = w + momentum * velocity - learning_rate * gradient
    past_velocit = velocity
    update_parameter
```

### 2.4.4 Back Propagation

앞의 알고리즘에서는 함수가 미분 가능하다고 가정하였기때문에 *derivative*를 직접 계산 할 수 있었음. 실제로 신경망은 많은 텐서 연산으로 구성되어있고 이 연산의 *derivative*는 잘 알려져있음.

3개의 텐서 연산 `a, b, c`와 가중치 행렬 `W1, W2, W3`로 구성된 네트워크 `f`가 있다고 가정한다면:
* `f(W1, W2, W3) = a(W1, b(W2, c(W3)))`

*Chain rule*을 신경망의 그래디언트 계산에 적용하여 만들어진 것이 **Backpropagation** 알고리즘
* Backpropagation은 최종 손실 값에서부터 시작
* 손실 값에 각 파라미터가 기여한 정도를 계산하기 위해 *chain rule*을 적용하여 최상위 층에서 하위층까지 거꾸로 진행